In [58]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import base64
import pandas as pd
import re
from openpyxl.utils import get_column_letter
import json

In [59]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai = OpenAI(api_key=OPENAI_API_KEY)

In [60]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [80]:
def get_response(image_path):
  base64_image = encode_image(image_path)
  response = openai.chat.completions.create(
      model="gpt-4o",
      temperature=0,
    messages=[
      {
        "role": "user", 
        "content": [
        {"type": "text", "text": "Analyze the image to accurately extract each individual's full name, job title, and associated company, and output as a JSON array of objects, with object containing 'Name,' 'Job Title,' and 'Company,' fields. Ignore all irrelevant text, icons, or visual elements."},
          {
            "type": "image_url", 
            "image_url": {
              "url": f"data:image/png;base64,{base64_image}"
            }
          }
        ]
      }
    ]
  )
  return response.choices[0].message.content


In [82]:
response=get_response("images/IMG_9396 1.PNG")
print(response)

```json
[
    {
        "Name": "Avi Richards",
        "Job Title": "CGO",
        "Company": "Ship Pros"
    },
    {
        "Name": "Barbara Reinish",
        "Job Title": "Brand Ambassador",
        "Company": "StarBars"
    },
    {
        "Name": "Brenee Staples",
        "Job Title": "Senior Regional Sales Director",
        "Company": "NetSuite"
    },
    {
        "Name": "Brianna Neufeld",
        "Job Title": "Co-founder/CEO",
        "Company": "Mate Wellness Inc."
    },
    {
        "Name": "Brianna Persall",
        "Job Title": "Brand Partnerships",
        "Company": "Creator.co"
    },
    {
        "Name": "Brittany & Jeff Didra",
        "Job Title": "Founder",
        "Company": "¡Wepa! Coffee Co."
    },
    {
        "Name": "Can Ozdoruk",
        "Job Title": "SVP, Head of Marketing",
        "Company": "Firework"
    }
]
```


In [62]:
def parse_table(data_text):
    """
    Parse a table from plain text into a list of dictionaries.
    
    Args:
        data_text (str): The text containing the table to parse
    
    Returns:
        list: A list of dictionaries containing parsed data
    """
    # Extract lines from the table
    cleaned_text = re.sub(r'^```json\s*|\s*```$', '', data_text.strip())

    # Parse the cleaned JSON text
    parsed_list = json.loads(cleaned_text)

    # Print the parsed list
    return(parsed_list)


In [63]:
def append_to_excel(parsed_data, file_name='companies.xlsx', sheet_name='Sheet1'):
    """
    Append data to an existing Excel file or create a new one, and adjust the column widths.
    
    Args:
        parsed_data (list): List of dictionaries containing the data to append
        file_name (str): Name of the Excel file
        sheet_name (str): Name of the sheet to append to
    """
    try:
        # Check if the file exists
        try:
            existing_df = pd.read_excel(file_name, sheet_name=sheet_name)
            file_exists = True
        except FileNotFoundError:
            existing_df = pd.DataFrame()
            file_exists = False
        
        # Convert parsed data to DataFrame
        new_df = pd.DataFrame(parsed_data)
        # Combine existing and new data if the file exists, otherwise use the new data alone
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        
        # Write combined data back to Excel
        with pd.ExcelWriter(file_name, engine='openpyxl', mode='a' if file_exists else 'w') as writer:
            # If the file exists, remove the old sheet if necessary
            workbook = writer.book
            if file_exists and sheet_name in workbook.sheetnames:
                idx = workbook.sheetnames.index(sheet_name)
                workbook.remove(workbook.worksheets[idx])
                
            # Write combined data to a new sheet
            combined_df.to_excel(writer, sheet_name=sheet_name, index=False)

            # Adjust column widths
            ws = workbook[sheet_name]
            for col in ws.columns:
                max_length = 0
                column = col[0].column_letter  # Get the column name
                for cell in col:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(cell.value)
                    except:
                        pass
                adjusted_width = (max_length + 2)  # Add extra space for padding
                ws.column_dimensions[column].width = adjusted_width

        print(f"Data has been appended to {file_name} and column widths adjusted.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Usage example
# Assuming 'output' is your input text containing the table
# parse_table(output)
# append_to_excel(parsed_data)

In [64]:
parsed_data=[]
files=["images/IMG_9395 1.PNG", "images/IMG_9396 1.PNG"]
for file in files:
    output=get_response(file)
    parsed_data.extend(parse_table(output))
append_to_excel(parsed_data)

Data has been appended to companies.xlsx and column widths adjusted.


In [49]:
# output=get_response("IMG_9402 1.PNG")
# print(output)

```json
[
    {
        "Name": "Mark Smith",
        "Job Title": "Co-Owner",
        "Company": "inVEND LLC"
    },
    {
        "Name": "Mat Drela",
        "Job Title": "CEO",
        "Company": "Retail Media Intelligence"
    },
    {
        "Name": "Meredith Martin",
        "Job Title": "Owner",
        "Company": "StarBars"
    },
    {
        "Name": "Natasha Bischoff",
        "Job Title": "Director of Sales & Marketing",
        "Company": "Marketing Support Services"
    },
    {
        "Name": "Nikki Loomba",
        "Job Title": "CEO",
        "Company": "Loomba Consulting Group, Inc."
    },
    {
        "Name": "Patti Regan",
        "Job Title": "CEO",
        "Company": "TRG Fulfillment"
    },
    {
        "Name": "Peter Misek",
        "Job Title": "CSO",
        "Company": "Retail Media Intelligence"
    }
]
```


In [195]:
print(parsed_data)

[{'Name': 'Adrienne Lufkin', 'Job Title': 'Founder', 'Company': 'Struesli'}, {'Name': 'Alberto Mayoral', 'Job Title': 'Sales & Education Executive', 'Company': 'Knesko Skin'}, {'Name': 'Alex Gabbert', 'Job Title': 'Brand Communications Director', 'Company': 'Publicity for Good'}, {'Name': 'Alineh Avanessian', 'Job Title': 'Influencer', 'Company': 'Alineh Inc.'}, {'Name': 'Allison Tryk', 'Job Title': 'Director of Sales', 'Company': 'Bloomkare.com'}, {'Name': 'Andrew Nicol', 'Job Title': 'Founder & CEO', 'Company': 'Element Brooklyn'}, {'Name': 'Avi Richards', 'Job Title': 'CGO', 'Company': 'Ship Pros'}, {'Name': 'Barbara Reinish', 'Job Title': 'Brand Ambassador', 'Company': 'StarBars'}, {'Name': 'Brenee Staples', 'Job Title': 'Senior Regional Sales Director', 'Company': 'NetSuite'}, {'Name': 'Brianna Neufeld', 'Job Title': 'Co-founder/CEO', 'Company': 'Mate Wellness Inc.'}, {'Name': 'Brianna Persall', 'Job Title': 'Brand Partnerships', 'Company': 'Creator.co'}, {'Name': 'Brittany & Jeff